# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [1]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In der Folge werden keine Regionen oder Gruppierungen von Ländern und nur die Jahre ab 1990 betrachtet. 
Von dem verbliebenen Datensatz werden nur Indikatoren behalten, die mindesten 20% gefüllt sind und nach ihrer Relevanz manuell ausgewählt wurden (siehe 03_preparing_data).

In [2]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base

In [3]:
base = reset_base()
base.isna().sum().sum()

170307

Um die Performanz unterschiedlicher Imutation Verfahren zu vergleichen werden weitere 1000 vorhandene (nicht NaN) Einträge entfernt. Diese 1000 Einträge werden später als Test-Daten verwendet, auf ihrer Grundlage lassen sich die Fehler der analysierten Verfahren errechnen. Um sicherzustellen, dass die Ergebnisse reproduzierbar sind wird ein Random State gesetzt. Dann werden zufällig Koordinaten zu Dateneinträgen gezogen. Da an dieser Stelle nur vorhandene Einträge relevant sind, werden zunächst zu viele Koordinaten gezogen, diese dann mit dem Datensatz abgeglichen und gelöscht, falls sie zu einem NaN zeigen und dann die ersten 1000 verbliebenen (und damit relevanten) Einträge ausgewählt. Diese werden in den Trainingsdaten entfernt. Auf diese Weise bleibt eine Reproduzierbarkeit erhalten.

In [4]:
base.head(3)

1990  1991  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1992  1993  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1994  1995  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1996  1997  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1998  1999  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 ...  \
Country Name Indicator Name                                      ...   
Afghanistan  Access to clean fuels and technologies for cook...  ...   
             Access to electricity (% of population)             ...   
             Access to electricity, rural (% of rural popula...  ...   

                                                                      2011  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  21.500000   
             Access to electricity (% of population)             43.222019   
             Access to electricity, rural (% of rural popula...  29.572881   

                                                                      2012  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  23.000000   
             Access to electricity (% of population)             69.100000   
             Access to electricity, rural (% of rural popula...  60.849157   

                                                                      2013  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  24.799999   
             Access to electricity (% of population)             68.982941   
             Access to electricity, rural (% of rural popula...  61.315788   

                                                                      2014  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  26.700001   
             Access to electricity (% of population)             89.500000   
             Access to electricity, rural (% of rural popula...  86.500512   

                                                                      2015  \
Country Name Indicator Name                                                  
Afgh

### Simulation fehlender Werte und Evaluation

In [5]:
def get_cords(frac):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(n)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

In [6]:
cords = get_cords(0.05)

Testdaten mit 5.0% fehlenden Werten (absolut: 8515)


In [7]:
#getting train data by changing randomly chosen values to NaN
def reset_train(cords):
    train = base.copy()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

In [8]:
results = []

In [9]:
def evaluate(method, df, t):
    
        
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train(cords)
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print('')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')
    
    results.append([method, r2, rmse, still_missing, t])


###  Imputation Verfahren

Es werden diese Imputation Methoden verglichen:
- Backcasting
- Durchschnitt
- regionaler Durchschnitt
- Iterative Imputer
- KNN Imputer

#### Backfill

In [10]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

In [11]:
base = reset_base()
train = reset_train(cords)



t0 = time.time()
df= impute_backfill(train) 
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/backfill.csv')
evaluate('Backfill', df, t)

Mit dieser Methode bleiben 32853 NaNs bestehen.

8256 Werte wurden für die Metriken verwendet.
r2: -16.124570911516813, rmse: 1.683781763968918


#### Durchschnitt des Indikators über alle Jahre hinweg

In [12]:
def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
        
    return df

In [13]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = impute_overall_means(train)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/mean.csv')
evaluate('Overall Mean', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: -2.231564589450377, rmse: 0.7202445836681456


#### Durchschnitt des Indikators für das jeweilige Jahr

In [14]:
def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

In [15]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = impute_yearly_means(train)
t1 = time.time()

t = t1-t0

evaluate('Yearly Mean',df, t)

Mit dieser Methode bleiben 52298 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: -0.09704256669343403, rmse: 0.41964815712332865


#### Regionaler Durchschnitt des Indikators für das jeweilige Jahr

In [16]:
def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

In [17]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = impute_yearly_means_per_region(train)
t1 = time.time()

t = t1-t0

evaluate('Yearly Mean per Region', df, t)

Mit dieser Methode bleiben 57262 NaNs bestehen.

8458 Werte wurden für die Metriken verwendet.
r2: -0.4349777575022391, rmse: 0.4815635503840548


#### Interpolation

In [18]:
def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate_all(df):
    df = df.interpolate()
    return df

In [19]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = interpolate3(train)
t1 = time.time()

t = t1-t0

evaluate('Interpolation 3', df, t)

Mit dieser Methode bleiben 32884 NaNs bestehen.

8235 Werte wurden für die Metriken verwendet.
r2: -2.959611964171229, rmse: 0.810691330925762


In [20]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = interpolate_all(train)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/interpolation.csv')
evaluate('Interpolation all', df, t)

Mit dieser Methode bleiben 58 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: -2.959674983506685, rmse: 0.7972661962988941


#### Iterative Imputer

An dieser Stelle wird der Iterative Imputer von SciKitLearn angewandt. Die drei Varianten laufen auf den drei Dimensionen des Datensatzes. 

In [21]:
def iterative_imputer_1(df):
    col = df.columns
    idx = df.index
    
    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)
    df= pd.DataFrame(df, columns=col, index=idx)
    return df

def iterative_imputer_2(df):
    df = df.unstack().T
    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

def iterative_imputer_3(df):

    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999, verbose=True)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [22]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = iterative_imputer_1(train)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/ice.csv')
evaluate('Iterative Imputer 1', df, t)

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9609051476884861, rmse: 0.07921973204963073


In [23]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = iterative_imputer_2(train)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/ice2.csv')
evaluate('Iterative Imputer 2', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9661514177508125, rmse: 0.07371295511229263


In [24]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = iterative_imputer_3(train)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/ice3.csv')
evaluate('Iterative Imputer 3', df, t)

[IterativeImputer] Completing matrix with shape (4898, 165)
[IterativeImputer] Change: 3.147511124721088e+16, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 1034671076716780.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 659123757395787.5, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 616363768648911.4, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 2396881809792168.5, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 603334623851478.6, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 3452555317561406.0, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 1566687283686987.5, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 562306276969730.8, scaled tolerance: 35084726045503.402 
[IterativeImputer] Change: 263990229283100.84, scaled tolerance: 35084726045503.402 


C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.662632017747099, rmse: 0.2327156083233093


#### MICE

In [25]:
def mice_imputer(df):
    n_imputations =  12
    dfs = []
    col = df.columns
    idx = df.index
    
    for i in range(n_imputations): 
        print(f'Imputation round {i}')
        iter_imp = IterativeImputer(random_state=i, sample_posterior=True, verbose=2)
        df_temp = iter_imp.fit_transform(df)
        dfs.append(df_temp)
    
    df = np.mean(np.array(dfs), axis=0)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

In [26]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = mice_imputer(train)
t1 = time.time()

t = t1-t0
df.to_csv('additional_data/imputed_sets/mice.csv')

Imputation round 0
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 22.13
[IterativeImputer] Ending imputation round 2/10, elapsed time 50.69
[IterativeImputer] Ending imputation round 3/10, elapsed time 74.83
[IterativeImputer] Ending imputation round 4/10, elapsed time 95.85
[IterativeImputer] Ending imputation round 5/10, elapsed time 125.55
[IterativeImputer] Ending imputation round 6/10, elapsed time 152.13
[IterativeImputer] Ending imputation round 7/10, elapsed time 175.06
[IterativeImputer] Ending imputation round 8/10, elapsed time 196.97
[IterativeImputer] Ending imputation round 9/10, elapsed time 219.44
[IterativeImputer] Ending imputation round 10/10, elapsed time 241.78
Imputation round 1
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 22.34
[IterativeImputer] Ending imputation round 2/10, elapsed time 43.44
[IterativeImputer] 

[IterativeImputer] Ending imputation round 8/10, elapsed time 119.59
[IterativeImputer] Ending imputation round 9/10, elapsed time 134.27
[IterativeImputer] Ending imputation round 10/10, elapsed time 149.29
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 14.57
[IterativeImputer] Ending imputation round 2/10, elapsed time 29.24
[IterativeImputer] Ending imputation round 3/10, elapsed time 43.59
[IterativeImputer] Ending imputation round 4/10, elapsed time 57.92
[IterativeImputer] Ending imputation round 5/10, elapsed time 72.18
[IterativeImputer] Ending imputation round 6/10, elapsed time 86.33
[IterativeImputer] Ending imputation round 7/10, elapsed time 100.59
[IterativeImputer] Ending imputation round 8/10, elapsed time 114.69
[IterativeImputer] Ending imputation round 9/10, elapsed time 128.86
[IterativeImputer] Ending imputation round 10/10, elapsed time 143.20


In [27]:
evaluate('MICE', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9588865414805373, rmse: 0.08123919037202011


#### KNN Imputer

In [28]:
def knn_imputer1(df):
    col = df.columns
    idx = df.index
    
    knn_imp = KNNImputer(n_neighbors=2)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)
    return df

def knn_imputer2(df, n):
    
    df = df.reset_index()
    df = df.set_index(['Indicator Name', 'Country Name'])
    df = df.unstack().T

    col = df.columns
    idx = df.index

    knn_imp = KNNImputer(n_neighbors=n)
    df= knn_imp.fit_transform(df)
    df = pd.DataFrame(df, columns=col, index=idx)

    df = df.unstack().T
    df = df.reset_index()
    df = df.set_index(['Country Name', 'Indicator Name'])
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [29]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = knn_imputer1(train)
t1 = time.time()

t = t1-t0

evaluate('KNN Imputer1', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.2944522579857045, rmse: 0.3365400724328245


In [31]:
base = reset_base()
train = reset_train(cords)

t0 = time.time()
df = knn_imputer2(train, 4)
t1 = time.time()

t = t1-t0

df.to_csv('additional_data/imputed_sets/knn.csv')
evaluate('KNN Imputer2', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

8515 Werte wurden für die Metriken verwendet.
r2: 0.9700605815538997, rmse: 0.06932585920533507


#### Results

In [32]:
results = pd.DataFrame(results, columns=['Methode', 'r2', 'RSME', 'Remaining NaNs', 'Time'])
results = results.set_index('Methode')

In [33]:
results

,r2,RSME,Remaining NaNs,Time
Methode,,,,
Backfill,-16.124571,1.683782,32853,0.000000
Overall Mean,-2.231565,0.720245,0,0.502221
Yearly Mean,-0.097043,0.419648,52298,0.219685
Yearly Mean per Region,-0.434978,0.481564,57262,0.329312
Interpolation 3,-2.959612,0.810691,32884,0.140671
Interpolation all,-2.959675,0.797266,58,0.094147
Iterative Imputer 1,0.960905,0.079220,0,15.893609
Iterative Imputer 2,0.966151,0.073713,0,30.838444
Iterative Imputer 3,0.662632,0.232716,0,105.027761
